In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys
sys.path.append("..")  # or "." if you're running from project root

from trader.unified import UnifiedTrader
import traceback



In [2]:
from trader.utils import TICK_SIZE, LOT_SIZE_STEP, snap2step


def snap_price(p):
    return snap2step(p, TICK_SIZE)


def snap_qty(q):
    return snap2step(q, LOT_SIZE_STEP)


def adjust_final_qty(quantities, target_total):
    actual_total = round(sum(quantities), 6)
    residual = round(target_total - actual_total, 6)
    if abs(residual) >= LOT_SIZE_STEP:
        quantities[-1] = snap_qty(quantities[-1] + residual)
    return quantities


In [3]:
from IPython.display import display, HTML

display(HTML('''
<style>
.output pre {
    font-size: 8px;
    font-family: monospace;
}
</style>
'''))

In [ ]:


########################################### TOP LEFT ######################################

exchange_dropdown = widgets.Dropdown(
    options=['testnet', 'binance', 'bitget'],
    value='bitget',
    description='CeX:',
    layout=widgets.Layout(width='80%'),
    style={'description_width': 'initial'}
)

connect_button = widgets.Button(
    description='Connect',
    button_style='success',
    layout=widgets.Layout(width='80%')
)

connect_output = widgets.Output()

def on_connect(b):
    with connect_output:
        clear_output()
        selected_exchange = exchange_dropdown.value
        print(f"Connecting to {selected_exchange}...")

        global trader
        trader = UnifiedTrader(exchange=selected_exchange.lower())
        trader.connect()

        symbol = "BTCUSDT"
        leverage_to_set = leverage_display.value

        # ✅ Set Bitget margin mode if applicable
        # if selected_exchange.lower() == "bitget":
        #     # trader.set_margin_mode(symbol=symbol, mode="crossed")
        #     trader.set_position_mode(symbol=symbol, mode="double")  # ✅ Add this


        # ✅ Push user-set leverage to exchange
        trader.set_leverage(symbol=symbol, leverage=leverage_to_set)

        # ✅ Pull updated balance only
        balance = trader.get_balance()

        # ✅ Update display fields
        balance_display.value = balance
        print(f"Connected ✅ Leverage set to {leverage_to_set}x")


connect_button.on_click(on_connect)

entry_fills = [
    widgets.Checkbox(value=True, description=str(i+1), layout=widgets.Layout(width='100px'))
    for i in range(4)
]


# ✅ SL fill checkboxes
sl_fills = [
    widgets.Checkbox(value=False, description=str(i+1), layout=widgets.Layout(width='100px'))
    for i in range(4)
]

fill_checkboxes_label = widgets.HTML("<b>Manual Fills</b>")

entry_fill_row = widgets.HBox(entry_fills, layout=widgets.Layout(gap='6px'))
sl_fill_row = widgets.HBox(sl_fills, layout=widgets.Layout(gap='6px'))




top_left = widgets.VBox(
    [
        exchange_dropdown,
        connect_button,
        connect_output,
        fill_checkboxes_label,
        entry_fill_row,
        sl_fill_row
    ],
    layout=widgets.Layout(min_height='100px', width='450px')
)


########################################### TOP MID ######################################

balance_display = widgets.FloatText(
    description='Balance (USDT)',
    value=0.0,
    disabled=False,
    layout=widgets.Layout(width='90%')
)

leverage_display = widgets.IntText(
    description='Leverage (x)',
    value=100,
    disabled=False,
    layout=widgets.Layout(width='90%')
)

position_direction = widgets.ToggleButtons(
    options=['Long', 'Short'],
    value='Long',
    description='Direction:',
    button_style='info',
    layout=widgets.Layout(width='100%')
)


top_mid = widgets.VBox(
    [balance_display, leverage_display, position_direction],
    layout=widgets.Layout(min_height='100px', width='250px')
)

########################################### TOP RIGHT ######################################

percentage_slider = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    step=1,
    description='% Position',
    continuous_update=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='260px')  # wider for fine control
)

margin_value = widgets.FloatText(
    description='Margin (USDT)',
    disabled=True,
    layout=widgets.Layout(width='200px')
)

size = widgets.FloatText(
    description='Size (USDT)',
    disabled=True,
    layout=widgets.Layout(width='200px')
)

qty_asset = widgets.FloatText(
    description='Qty (Asset)',
    disabled=True,
    layout=widgets.Layout(width='200px')
)

sl_qty_total = widgets.FloatText(
    description='SL Qty',
    disabled=True,
    layout=widgets.Layout(width='200px')
)

remaining_qty_display = widgets.FloatText(
    description='Rem Qty',
    disabled=True,
    layout=widgets.Layout(width='200px')
)

tp_qty_display = widgets.FloatText(
    description='TP Qty',
    disabled=True,
    layout=widgets.Layout(width='200px')
)


def update_text(change):
    percent = percentage_slider.value
    margin_value.value = (percent / 100) * balance_display.value
    size.value = margin_value.value * leverage_display.value




percentage_slider.observe(update_text, names='value')



top_right = widgets.VBox(
    [
        percentage_slider,
        margin_value,
        size,
        qty_asset,
        sl_qty_total,
        remaining_qty_display,
        tp_qty_display  # ✅ New line
    ],
    layout=widgets.Layout(min_height='100px', width='360px')
)



########################################### COMBINE TOP ROW ######################################

top_row = widgets.HBox(
    [top_left, top_mid, top_right],
    layout=widgets.Layout(justify_content='space-between', width='100%')
)



########################################### BOT LEFT ######################################
entry_heading = widgets.HTML("<h4 style='margin-bottom:5px;'>Entry Ladder</h4>")

lock_entry_pcts_checkbox = widgets.Checkbox(
    value=True,
    description='%',
    indent=False,
    layout=widgets.Layout(width='80px')
)


sl_mode_dropdown = widgets.Dropdown(
    options=["reverse", "same", "custom"],
    value="reverse",
    description="SL",
    layout=widgets.Layout(width='150px')
)


entry_heading_row = widgets.HBox([
    # entry_heading,
    lock_entry_pcts_checkbox,
    sl_mode_dropdown
], layout=widgets.Layout(justify_content='space-between', width='100%'))


entry_price_A = widgets.FloatText(
    description="",
    layout=widgets.Layout(width='80px')
)

entry_price_B = widgets.FloatText(
    description='',
    layout=widgets.Layout(width='80px')
)

avg_entry_display = widgets.FloatText(
    description='',
    disabled=True,
    layout=widgets.Layout(width='80px')
)


# Wrap in horizontal layout
entry_panel = widgets.HBox(
    [entry_price_A, entry_price_B, avg_entry_display],
    layout=widgets.Layout(justify_content='space-between', overflow='visible', width='100%', gap='10px')
)

# Also update qty/size when entry prices change
entry_price_A.observe(update_text, names='value')
entry_price_B.observe(update_text, names='value')

entry_label_row = widgets.HBox([
    widgets.Label("Price", layout=widgets.Layout(width='80px')),
    widgets.Label("%", layout=widgets.Layout(width='40px')),
    widgets.Label("USD", layout=widgets.Layout(width='60px')),
    widgets.Label("Qty", layout=widgets.Layout(width='80px')),
])


# Price ladder (4 steps)
ladder_prices = [widgets.FloatText(description=f"", disabled=True, layout=widgets.Layout(width='80px')) for i in range(4)]
ladder_percentages = [widgets.IntText(value=val, layout=widgets.Layout(width='40px')) for val in [10, 20, 30, 40]]
# New: calculated amount in USDT per level
ladder_usdt_amounts = [
    widgets.FloatText(disabled=True, layout=widgets.Layout(width='60px')) for _ in range(4)
]

# New: calculated asset amount per level (e.g., BTC)
ladder_asset_amounts = [
    widgets.FloatText(disabled=True, layout=widgets.Layout(width='80px')) for _ in range(4)
]

# Arrange rows: each row has [price display | percentage input]
ladder_rows = [
    widgets.HBox([
        ladder_prices[i],
        ladder_percentages[i],
        ladder_usdt_amounts[i],
        ladder_asset_amounts[i]
    ], layout=widgets.Layout(gap='8px'))
    for i in range(4)
]

send_entry_button = widgets.Button(
    description='Send Entry + SL',
    button_style='warning',
    layout=widgets.Layout(width='200px')
)

entry_output = widgets.Output()



for p in ladder_prices + ladder_percentages:
    p.observe(update_text, names='value')


def update_avg_entry_price():
    weighted_sum = 0
    total_percent = 0

    for i in range(4):
        price = ladder_prices[i].value
        percent = ladder_percentages[i].value
        if price and percent:
            weighted_sum += price * percent
            total_percent += percent

    avg_price = weighted_sum / total_percent if total_percent else 0
    avg_entry_display.value = round(avg_price, 2)
    return avg_price


def update_ladder_prices(change=None):
    a = entry_price_A.value
    b = entry_price_B.value
    steps = 3
    total_size = size.value

    total_qty = 0

    for i in range(4):
        # ✅ Snap price for each level
        raw_price = a + i * (b - a) / steps
        price = snap_price(raw_price)
        ladder_prices[i].value = price

        # Allocate % of size in USDT
        percent = ladder_percentages[i].value
        usdt_alloc = (percent / 100) * total_size
        ladder_usdt_amounts[i].value = round(usdt_alloc, 2)

        # ✅ Snap quantity
        qty = snap_qty(usdt_alloc / price) if price else 0
        ladder_asset_amounts[i].value = qty

        total_qty += qty

    # ✅ Update Qty (Asset)
    qty_asset.value = total_qty

    # ✅ Update avg entry price using dedicated helper
    update_avg_entry_price()


def toggle_entry_percentages(change=None):
    locked = lock_entry_pcts_checkbox.value
    for pct_input in ladder_percentages:
        pct_input.disabled = locked

lock_entry_pcts_checkbox.observe(toggle_entry_percentages, names='value')
toggle_entry_percentages()

# Trigger updates on relevant changes
percentage_slider.observe(update_ladder_prices, names='value')
size.observe(update_ladder_prices, names='value')
balance_display.observe(update_ladder_prices, names='value')
leverage_display.observe(update_ladder_prices, names='value')

# Also attach to individual percent fields
for pct_input in ladder_percentages:
    pct_input.observe(update_ladder_prices, names='value')

# Observe both price inputs
entry_price_A.observe(update_ladder_prices, names='value')
entry_price_B.observe(update_ladder_prices, names='value')

# Initial update

update_ladder_prices()
update_text(None)


# Ladder layout block (can go into bot_mid or wherever you want)
ladder_panel = widgets.VBox(ladder_rows)

# Display if testing


bot_left = widgets.VBox(
    [
        entry_heading_row,
        entry_panel,
        entry_label_row,
        ladder_panel,
        send_entry_button,   # ✅ Add here
        entry_output         # ✅ Output feedback right below button
    ],
    layout=widgets.Layout(min_height='300px', width='280px')
)


########################################### BOT MID ######################################
total_tp_profit = widgets.FloatText(
    description='P',
    disabled=True,
    layout=widgets.Layout(width='80px'),
    style={'description_width': 'initial'}
)


sl_heading_label = widgets.HTML("<h4 style='margin-bottom:5px;'>Stop Loss Ladder</h4>")
sl_header_bar = widgets.HBox(
    [sl_heading_label, total_tp_profit],
    layout=widgets.Layout(justify_content='space-between', width='100%')
)


sl_price_A = widgets.FloatText(description='', layout=widgets.Layout(width='80px'))
sl_price_B = widgets.FloatText(description='', layout=widgets.Layout(width='80px'))

total_sl_loss = widgets.FloatText(
    description='L',
    disabled=True,
    layout=widgets.Layout(width='80px'),
    style={'description_width': 'initial'}
)

sl_label_row = widgets.HBox([
    widgets.Label("Price", layout=widgets.Layout(width='80px')),
    widgets.Label("%", layout=widgets.Layout(width='40px')),
    widgets.Label("L (USD)", layout=widgets.Layout(width='60px')),
    widgets.Label("Qty", layout=widgets.Layout(width='80px')),
])

sl_price_inputs = widgets.HBox(
    [sl_price_A, sl_price_B, total_sl_loss],
    layout=widgets.Layout(justify_content='space-between', width='100%', gap='10px')
)

sl_prices = [widgets.FloatText(disabled=True, layout=widgets.Layout(width='80px')) for _ in range(4)]
sl_percentages = [widgets.IntText(value=val, layout=widgets.Layout(width='40px')) for val in [40, 30, 20, 10]]
sl_usdt_loss = [widgets.FloatText(disabled=True, layout=widgets.Layout(width='60px')) for _ in range(4)]
sl_closed_qty = [widgets.FloatText(disabled=True, layout=widgets.Layout(width='80px')) for _ in range(4)]

sl_rows = [
    widgets.HBox([
        sl_prices[i],
        sl_percentages[i],
        sl_usdt_loss[i],
        sl_closed_qty[i]
    ], layout=widgets.Layout(gap='8px'))
    for i in range(4)
]


def update_sl_panel(change=None):
    mode = sl_mode_dropdown.value  # "same", "reverse", "custom"
    a = sl_price_A.value
    b = sl_price_B.value
    direction = position_direction.value
    total_qty = qty_asset.value
    steps = 3

    # Step 1: Interpolate and snap SL prices
    for i in range(4):
        sl_prices[i].value = snap_price(a + i * (b - a) / steps)

    # Step 2: Determine SL quantities based on mode
    if mode == "same":
        # Lock percentages to match ladder_percentages
        for i in range(4):
            sl_percentages[i].value = ladder_percentages[i].value
            sl_percentages[i].disabled = True

        qty_list = [ladder_asset_amounts[i].value for i in range(4)]

    elif mode == "reverse":
        # Lock percentages to match reversed ladder_percentages
        for i in range(4):
            sl_percentages[i].value = ladder_percentages[3 - i].value
            sl_percentages[i].disabled = True

        qty_list = [ladder_asset_amounts[i].value for i in reversed(range(4))]

    else:  # "custom" mode
        for i in range(4):
            sl_percentages[i].disabled = False

        sl_pcts = [p.value for p in sl_percentages]
        total_pct = sum(sl_pcts)
        raw_qtys = [(pct / total_pct) * total_qty for pct in sl_pcts]

        # Snap + adjust remainder to last
        snapped_qtys = []
        total_snapped = 0
        for q in raw_qtys:
            s = snap_qty(q)
            snapped_qtys.append(s)
            total_snapped += s

        diff = round(total_qty - total_snapped, 6)
        if abs(diff) >= LOT_SIZE_STEP:
            snapped_qtys[-1] = snap_qty(snapped_qtys[-1] + diff)

        qty_list = snapped_qtys

    # Step 3: Apply quantities and compute losses
    for i in range(4):
        qty = qty_list[i]
        sl_closed_qty[i].value = qty

        sl = sl_prices[i].value
        entry = avg_entry_display.value  # Use average!

        if direction == 'Long':
            loss = max(entry - sl, 0) * qty
        else:
            loss = max(sl - entry, 0) * qty

        sl_usdt_loss[i].value = round(loss, 4)
        sl_usdt_loss[i].style.text_color = 'red' if loss > 0 else 'gray'

    # Final totals
    total_sl_loss.value = round(sum(sl_usdt_loss[i].value for i in range(4)), 4)
    sl_qty_total.value = round(sum(qty_list), 6)




sl_price_A.observe(update_sl_panel, names='value')
sl_price_B.observe(update_sl_panel, names='value')
position_direction.observe(update_sl_panel, names='value')
size.observe(update_sl_panel, names='value')
sl_mode_dropdown.observe(update_sl_panel, names='value')

for p in sl_percentages:
    p.observe(update_sl_panel, names='value')
for ladder_price in ladder_prices:
    ladder_price.observe(update_sl_panel, names='value')

tp_output = widgets.Output()

send_tp_button = widgets.Button(
    description='Send TPs',
    button_style='info',
    layout=widgets.Layout(width='200px')
)
use_limit_tp_checkbox = widgets.Checkbox(
    value=False,
    description='Limit',
    indent=False,
    layout=widgets.Layout(width='140px')
)
send_tp_row = widgets.HBox(
    [send_tp_button, use_limit_tp_checkbox],
    layout=widgets.Layout(justify_content='space-between')
)

bot_mid = widgets.VBox(
    [sl_header_bar, sl_price_inputs, sl_label_row] + sl_rows + [send_tp_row, tp_output],
    layout=widgets.Layout(min_height='360px', width='280px')
)


def on_send_entry_click(b):
    with entry_output:
        clear_output()

        if not trader:
            print("❌ Not connected to an exchange.")
            return

        direction = position_direction.value  # "Long" or "Short"

        print("📥 Sending ENTRY orders...")

        for i in range(4):
            price = ladder_prices[i].value
            qty = ladder_asset_amounts[i].value

            if price > 0 and qty > 0:
                try:
                    trader.place_entry_order(
                        direction=direction,
                        price=price,
                        quantity=qty
                    )
                    print(f"✅ Entry {i+1}: {direction} {qty} @ {price}")
                except Exception as e:
                    print(f"❌ Entry {i+1} failed:", e)

        print("\n🛑 Sending STOP-LOSS orders...")

        for i in range(4):
            sl_price = sl_prices[i].value
            qty = sl_closed_qty[i].value

            if sl_price > 0 and qty > 0:
                try:
                    trader.place_sl_order(
                        price=sl_price,
                        quantity=qty,
                        direction=direction
                    )
                    print(f"✅ SL {i+1}: {direction} SL {qty} @ {sl_price}")
                except Exception as e:
                    print(f"❌ SL {i+1} failed:", e)


send_entry_button.on_click(on_send_entry_click)


########################################### BOT RIGHT ######################################
tp_heading = widgets.HBox([
    widgets.HTML("<h4 style='margin-bottom:5px;'>Take Profits</h4>"),
    widgets.Button(description="Suggest TP", button_style="info", layout=widgets.Layout(width='120px'))
])
tp_suggest_button = tp_heading.children[1]
# TP Panel - 7 rows max
tp_label_row = widgets.HBox([
    widgets.Label("Price", layout=widgets.Layout(width='80px')),
    widgets.Label("%", layout=widgets.Layout(width='40px')),
    widgets.Label("P (USD)", layout=widgets.Layout(width='60px')),
    widgets.Label("Qty", layout=widgets.Layout(width='80px')),
])

tp_prices = [widgets.FloatText(placeholder='TP', step=100.0, layout=widgets.Layout(width='80px')) for _ in range(7)]
tp_percentages = [widgets.IntText(value=val, layout=widgets.Layout(width='40px')) for val in [20, 18, 0, 0, 0, 0, 0]]
tp_profits = [widgets.FloatText(disabled=True, layout=widgets.Layout(width='60px')) for _ in range(7)]
tp_closed_qty = [widgets.FloatText(disabled=True, layout=widgets.Layout(width='80px')) for _ in range(7)]

tp_rows = [
    widgets.HBox([
        tp_prices[i],
        tp_percentages[i],
        tp_profits[i],
        tp_closed_qty[i]
    ], layout=widgets.Layout(gap='8px'))
    for i in range(7)
]

def update_tp_panel(change=None):
    direction = position_direction.value
    total_qty = remaining_qty_display.value
    avg_entry = avg_entry_display.value

    for i in range(7):
        pct = tp_percentages[i].value / 100
        tp_price = tp_prices[i].value

        # Quantity is % of total, snapped
        qty = snap_qty(total_qty * pct)
        tp_closed_qty[i].value = qty

        # Profit based on direction
        if direction == 'Long':
            profit = max(tp_price - avg_entry, 0) * qty
        else:
            profit = max(avg_entry - tp_price, 0) * qty

        tp_profits[i].value = round(profit, 4)
        tp_profits[i].style.text_color = 'green' if profit > 0 else 'gray'

    total_profit = sum(tp_profits[i].value for i in range(7))
    total_tp_profit.value = round(total_profit, 4)
    tp_qty_total = sum(tp_closed_qty[i].value for i in range(7))
    tp_qty_display.value = round(tp_qty_total, 6)






position_direction.observe(update_tp_panel, names='value')
size.observe(update_tp_panel, names='value')
balance_display.observe(update_tp_panel, names='value')
leverage_display.observe(update_tp_panel, names='value')
# leverage_display.observe(update_text, names='value')
# leverage_display.observe(update_avg_entry_price, names='value')
# leverage_display.observe(update_sl_panel, names='value')
# leverage_display.observe(update_tp_panel, names='value')

for w in tp_prices + tp_percentages:
    w.observe(update_tp_panel, names='value')
for ladder_price in ladder_prices:
    ladder_price.observe(update_tp_panel, names='value')




def suggest_tp_prices(b=None):
    try:
        total_loss = total_sl_loss.value
        total_margin = margin_value.value
        total_qty = qty_asset.value
        avg_entry = avg_entry_display.value
        is_long = position_direction.value == 'Long'

        # TP1: Cover SL loss using first TP %
        pct1 = tp_percentages[0].value / 100
        qty1 = total_qty * pct1
        profit_per_unit_1 = total_loss / qty1 if qty1 else 0
        tp1 = avg_entry + profit_per_unit_1 if is_long else avg_entry - profit_per_unit_1
        tp_prices[0].value = snap_price(tp1)

        # TP2: Cover margin using second TP %
        pct2 = tp_percentages[1].value / 100
        qty2 = total_qty * pct2
        profit_per_unit_2 = total_margin / qty2 if qty2 else 0
        tp2 = avg_entry + profit_per_unit_2 if is_long else avg_entry - profit_per_unit_2
        tp_prices[1].value = snap_price(tp2)

        # TP3–TP7: Increment by step (80% of TP2 unit profit)
        step = profit_per_unit_2 * 0.8
        for i in range(2, 7):
            prev = tp_prices[i - 1].value
            next_tp = prev + step if is_long else prev - step
            tp_prices[i].value = snap_price(next_tp)

        update_tp_panel()

    except Exception as e:
        print("❌ Error in suggest_tp_prices:", e)


tp_suggest_button.on_click(suggest_tp_prices)


def on_send_tp_click(b):
    with tp_output:
        clear_output()

        if not trader:
            print("❌ Not connected to an exchange.")
            return

        direction = position_direction.value  # "Long" or "Short"

        print("🎯 Sending TAKE PROFIT orders...")

        for i in range(7):
            tp_price = tp_prices[i].value
            qty = tp_closed_qty[i].value

            if tp_price > 0 and qty > 0:
                try:
                    if use_limit_tp_checkbox.value:  # ✅ default is True
                        trader.place_limit_tp_order(
                            price=tp_price,
                            quantity=qty,
                            direction=direction
                        )
                    else:
                        trader.place_tp_order(
                            price=tp_price,
                            quantity=qty,
                            direction=direction
                        )
                    print(f"✅ TP {i+1}: {direction} TP {qty} @ {tp_price}")
                except Exception as e:
                    print(f"❌ TP {i+1} failed:", e)


send_tp_button.on_click(on_send_tp_click)

def update_rem_qty_and_tp_panel(change=None):
    entry_total = sum(
        ladder_asset_amounts[i].value for i in range(4)
        if entry_fills[i].value
    )
    sl_total = sum(
        sl_closed_qty[i].value for i in range(4)
        if sl_fills[i].value
    )
    rem = entry_total - sl_total
    rem = max(rem, 0)
    remaining_qty_display.value = snap_qty(rem)

    # ✅ Refresh TP calculations using this remaining qty
    update_tp_panel()


# Observe checkboxes
for chk in entry_fills + sl_fills:
    chk.observe(update_rem_qty_and_tp_panel, names='value')

# Observe all entry ladder amounts
for w in ladder_asset_amounts:
    w.observe(update_rem_qty_and_tp_panel, names='value')

# Observe all SL quantities
for w in sl_closed_qty:
    w.observe(update_rem_qty_and_tp_panel, names='value')

update_rem_qty_and_tp_panel()

bot_right = widgets.VBox(
    [tp_heading, tp_label_row] + tp_rows,
    layout=widgets.Layout(min_height='300px', width='280px')
)


bot_row = widgets.HBox(
    [bot_left, bot_mid, bot_right],
    layout=widgets.Layout(justify_content='space-between', width='100%')
)


# Display
display(top_row,bot_row)
